In [1]:
import numpy as np
import pandas as pd

In [2]:
def print_results(setting, agents=None):
    """ Print Results from Learning and Simulation

    Args:
        setting (str): setting (single_item, llg, ...)
    """

    agent_specified = agents is not None

    # import data
    log_learn = pd.read_csv(f"../log/{setting}/log_learn_agg.csv")
    log_learn_all = pd.read_csv(f"../log/{setting}/log_learn.csv")

    print(f"Experiment".ljust(20), f"Learner".ljust(15), f"Agent".rjust(5), f"Utility Loss".rjust(15), f"L2-norm".rjust(15), f"Running Time".rjust(15),f"Conv.".rjust(8), f"Util. Loss Discr.".rjust(20),)
    print("-"*120)

    for exp in log_learn.experiment.unique():

        if not agent_specified:
            agents_list = log_learn[(log_learn.experiment == exp)].agent.unique()
            ignore_experiment = False

        elif agents not in log_learn[(log_learn.experiment == exp)].agent.unique():
            ignore_experiment = True
        else:
            agents_list = [agents]
            ignore_experiment = False

        if not ignore_experiment:           

            for learner in log_learn[log_learn.experiment==exp].learner.unique():            
                for agent in agents_list:
                    if setting not in ['contest_game']:    
                        # simulation
                        log_sim = pd.read_csv(f"../log/{setting}/log_sim_agg.csv")
                        mean_util_loss = log_sim[(log_sim.experiment == exp)&(log_sim.learner == learner)&(log_sim.agent==agent)&(log_sim.metric=="util_loss")]['mean'].item()
                        std_util_loss = log_sim[(log_sim.experiment == exp)&(log_sim.learner == learner)&(log_sim.agent==agent)&(log_sim.metric=="util_loss")]['std'].item()
                        
                        if setting != 'split_award':
                            mean_l2 = log_sim[(log_sim.experiment == exp)&(log_sim.learner == learner)&(log_sim.agent==agent)&(log_sim.metric=="l2_norm")]['mean'].item()
                            std_l2 = log_sim[(log_sim.experiment == exp)&(log_sim.learner == learner)&(log_sim.agent==agent)&(log_sim.metric=="l2_norm")]['std'].item()
                        else:
                            mean_l2 = std_l2 = np.nan

                    else:
                        mean_util_loss = std_util_loss = np.nan
                        mean_l2 = std_l2 = np.nan                

                    # learning
                    init_time =  log_learn_all[(log_learn_all.experiment==exp)&(log_learn_all.learner==learner)&(log_learn_all.agent==agent)].time_init.max()
                    run_time_min = log_learn_all[(log_learn_all.experiment==exp)&(log_learn_all.learner==learner)&(log_learn_all.agent==agent)].time_run.min()
                    run_time_max = log_learn_all[(log_learn_all.experiment==exp)&(log_learn_all.learner==learner)&(log_learn_all.agent==agent)].time_run.max()
        
                    number_conv = log_learn_all[(log_learn_all.experiment==exp)&(log_learn_all.learner==learner)&(log_learn_all.agent==agent)].convergence.sum()
                    number_exp = log_learn_all[(log_learn_all.experiment==exp)&(log_learn_all.learner==learner)&(log_learn_all.agent==agent)].convergence.count()
                    convergence = f"{number_conv:n}/{number_exp:n}"
                    mean_util_loss_discr = log_learn[(log_learn.experiment == exp)&(log_learn.learner == learner)&(log_learn.agent==agent)&(log_learn.Metric=="utility_loss")]['Mean'].item()
                    std_util_loss_discr = log_learn[(log_learn.experiment == exp)&(log_learn.learner == learner)&(log_learn.agent==agent)&(log_learn.Metric=="utility_loss")]['Std'].item()

                    
                    # print results
                    print(
                        f"{exp}".ljust(20),f"{learner}".ljust(15),f"{agent}".rjust(5),
                        f"{mean_util_loss:.3f} ({std_util_loss:.3f})".rjust(15),
                        f"{mean_l2:.3f} ({std_l2:.3f})".rjust(15), 
                        f"{init_time+run_time_min:3.1f}-{init_time+run_time_max:3.1f}".rjust(15),
                        convergence.rjust(8),
                        f"{mean_util_loss_discr:.3f} ({std_util_loss_discr:.3f})".rjust(20),
                        )
            print("-"*120) 

In [3]:
print_results('single_item')

Experiment           Learner         Agent    Utility Loss         L2-norm    Running Time    Conv.    Util. Loss Discr.
------------------------------------------------------------------------------------------------------------------------
affiliated_values    soda_entro_av       1   0.002 (0.000)   0.014 (0.000)       15.2-15.8     0/10        0.004 (0.000)
affiliated_values    soda_eucl_av        1   0.002 (0.000)   0.012 (0.000)       11.3-11.9     0/10        0.004 (0.000)
affiliated_values    soma_eucl_av        1   0.002 (0.000)   0.014 (0.000)       11.3-11.4     0/10        0.001 (0.000)
affiliated_values    sofw_std            1   0.004 (0.001)   0.020 (0.002)       11.2-11.2     0/10        0.001 (0.000)
affiliated_values    fp                  1   0.005 (0.000)   0.025 (0.001)       11.8-12.8     0/10        0.003 (0.000)
------------------------------------------------------------------------------------------------------------------------
common_value         soma_eucl_c

In [4]:
print_results('llg_auction', 'L')

Experiment           Learner         Agent    Utility Loss         L2-norm    Running Time    Conv.    Util. Loss Discr.
------------------------------------------------------------------------------------------------------------------------
nb_gamma1            soda_eucl           L   0.001 (0.000)   0.013 (0.000)         1.0-1.1    10/10        0.000 (0.000)
nb_gamma1            soma_eucl           L   0.000 (0.000)   0.012 (0.000)         0.9-0.9    10/10        0.000 (0.000)
nb_gamma1            sofw_std            L   0.000 (0.000)   0.013 (0.001)         1.6-2.1    10/10        0.000 (0.000)
nb_gamma1            soda_entro          L   0.001 (0.000)   0.014 (0.001)        8.9-15.3    10/10        0.000 (0.000)
nb_gamma1            fp                  L   0.001 (0.000)   0.017 (0.000)       31.4-38.7     0/10        0.000 (0.000)
------------------------------------------------------------------------------------------------------------------------
nb_gamma2            soda_eucl  

In [11]:
print_results('split_award')

Experiment           Learner         Agent    Utility Loss         L2-norm    Running Time    Conv.    Util. Loss Discr.
------------------------------------------------------------------------------------------------------------------------
sa_gaussian          soda_entro_gaus     1  -0.023 (0.000)       nan (nan)       58.6-64.7    10/10        0.000 (0.000)
sa_gaussian          soda_eucl_gaus      1  -0.069 (0.001)       nan (nan)       68.7-72.1    10/10        0.000 (0.000)
sa_gaussian          sofw_std            1  -0.002 (0.067)       nan (nan)     418.3-421.1     0/10        0.000 (0.000)
sa_gaussian          fp                  1   0.065 (0.070)       nan (nan)     417.4-424.1     0/10        0.008 (0.001)
sa_gaussian          soma_eucl_gaus      1  -0.082 (0.003)       nan (nan)     416.1-424.4     0/10        0.002 (0.000)
------------------------------------------------------------------------------------------------------------------------
sa_uniform           soda_entro_

In [12]:
416/60

6.933333333333334

In [9]:
print_results('contest_game', 'strong')

Experiment           Learner         Agent    Utility Loss         L2-norm    Running Time    Conv.    Util. Loss Discr.
------------------------------------------------------------------------------------------------------------------------
asym_ratio1          soda_entro      strong       nan (nan)       nan (nan)         0.3-0.4    10/10        0.000 (0.000)
asym_ratio1          soda_eucl       strong       nan (nan)       nan (nan)         0.0-0.0    10/10        0.000 (0.000)
asym_ratio1          soma_eucl       strong       nan (nan)       nan (nan)         0.0-0.1    10/10        0.000 (0.000)
asym_ratio1          sofw_std        strong       nan (nan)       nan (nan)         0.0-0.1    10/10        0.000 (0.000)
------------------------------------------------------------------------------------------------------------------------
asym_ratio2          soda_entro      strong       nan (nan)       nan (nan)         0.7-1.7    10/10        0.000 (0.000)
asym_ratio2          soda_e